# Query Notebook

In [ ]:
from math import radians, sin, cos, sqrt, atan2

import pandas as pd

In [ ]:
SAMPLE = False

In [ ]:
# Load business data
with open(f"data/02_postprocessing/{'sample/' if SAMPLE else ''}businesses.csv") as f:
    businesses = pd.read_csv(f, header=0)
# Load user data
with open(f"data/02_postprocessing/{'sample/' if SAMPLE else ''}users.csv") as f:
    users = pd.read_csv(f, header=0)
# Load friends data
with open(f"data/02_postprocessing/{'sample/' if SAMPLE else ''}friends.csv") as f:
    friends = pd.read_csv(f, header=0)
# Load ratings data
with open(f"data/02_postprocessing/{'sample/' if SAMPLE else ''}ratings.csv") as f:
    ratings = pd.read_csv(f, header=0)

In [ ]:
# The user query. Can be changed and rerun from this cell for a new query
# Change the userID to change the center of the friend network
# Change the location to get new results
# The literals set minimum rating and maximum distance
query = {
    "user": "4ZaqBJqt7laPPs8xfWvr6A",
    "location": (41.5074, 81.6096),
    "type": "restaurants",
    "keywords": ["abby", "doctors", "target", "shopping", "fashion", "nails", "bars", "food"],
    "literals": {"distance" : 100, "rating" : 1},
    "k": 5
}

Program flow

1. Query the database for a list of locations that match the type of location
2. Narrow down the list to make sure they match the literals
3. Match keywords to the location
4. Apply the friend rating filter
5. Return K results

# Literal Processing

When considering the literal processing, this creates an new set that narrows down the amount of businesses that are going to be chosen by its 'literal' or global attribute (distance by radius & global rating number)

In [ ]:
# calculating the difference between two different points of lat & long
# units = km
def dist(lat1, long1, lat2, long2):
    earthRadius = 6371.0

    # converting the latitude and longitude from degrees to radians
    lat1 = radians(lat1)
    long1 = radians(long1)
    lat2 = radians(lat2)
    long2 = radians(long2)

    # finding the distance between respective latitudes and longitudes
    latDist = lat2 - lat1
    longDist = long2 - long1

    # use of the haversine formula
    a = sin(latDist / 2)**2 + cos(lat1) * cos(lat2) * sin(longDist / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Calculate the distance
    distance = earthRadius * c

    return distance

In [ ]:
# Get distances
businesses["distance"] = businesses.apply(lambda row: dist(query["location"][0], query["location"][1], row["latitude"], row["longitude"]), axis = 1)
businesses["distance"] = businesses["distance"] / 100 # convert from meters to km
businesses.head()

In [ ]:
# Check rating
businesses["rating_check"] = businesses["stars"] >= query["literals"]["rating"]
businesses.head()

In [ ]:
# Combine literals
businesses["literals"] = businesses["rating_check"] & (businesses["distance"] <= query["literals"]["distance"])
businesses[list(set(businesses.columns) - {"distance", "rating_check"})].loc[businesses["literals"]].head()

# Type Matching

In [ ]:
# Make type lower case
businesses["type"] = businesses["type"].str.lower()
query["type"] = query["type"].lower()

In [ ]:
businesses["type_check"] = businesses["type"] == query["type"]
businesses.loc[businesses["type_check"]].head()

# Weighted Rating Average

In [ ]:
first_degree_multiplier = 13
second_degree_multiplier = 8
third_degree_multiplier = 5

In [ ]:
# Add a multiplier column to the ratings table
ratings["multiplier"] = 1

In [ ]:
# Find the users that are first degree connections
first_degree = friends.loc[friends["user_id"] == query["user"]]["friend_id"]
ratings.loc[ratings["user_id"].isin(first_degree), "multiplier"] = first_degree_multiplier

In [ ]:
# Find second degree connections
second_degree = friends.loc[friends["user_id"].isin(first_degree)]["friend_id"]
# Set ratings to second degree multiplier if it is set to 1
ratings.loc[ratings["user_id"].isin(second_degree) & (ratings["multiplier"] == 1), "multiplier"] = second_degree_multiplier

In [ ]:
# Find third degree connections
third_degree = friends.loc[friends["user_id"].isin(second_degree)]["friend_id"]
# Set ratings to third degree multiplier if it is set to 1
ratings.loc[ratings["user_id"].isin(third_degree) & (ratings["multiplier"] == 1), "multiplier"] = third_degree_multiplier

In [ ]:
ratings["multiplied_ratings"] = ratings["norm_std_rating"] * ratings["multiplier"]

In [ ]:
# Give each business a score based on the ratings and their multipliers
business_ratings = businesses[["business_id"]].merge(ratings[["business_id", "multiplied_ratings", "multiplier"]], on="business_id", how="left")
business_ratings.head()

In [ ]:
business_ratings_aggregated = business_ratings.groupby("business_id").sum()
business_ratings_aggregated["score"] = (
    business_ratings_aggregated["multiplied_ratings"]
    / business_ratings_aggregated["multiplier"]
)
business_ratings_aggregated.reset_index(inplace=True)

# Normalize the scores
min_score = business_ratings_aggregated["score"].min()
max_score = business_ratings_aggregated["score"].max()
business_ratings_aggregated["norm_score"] = (
    business_ratings_aggregated["score"] - min_score
) / (max_score - min_score)

business_ratings_aggregated.head()

In [ ]:
# Merge with the original business data
businesses = businesses.merge(business_ratings_aggregated[["business_id", "norm_score"]], on="business_id", how="left")
businesses.head()

# Keyword Matching

In [ ]:
# read businesses_keywords.csv file
with open("data/02_postprocessing/businesses_keywords.csv") as f:
    business_keywords_df = pd.read_csv(f, header=0)

In [ ]:
keywords = query["keywords"]
# Filter businesses based on keywords
businesses_filtered = business_keywords_df[business_keywords_df["keyword"].isin(keywords)]
# Count the number of keywords matched
businesses_filtered_count = businesses_filtered.groupby("business_id").count().reset_index()
# Rename the column
businesses_filtered_count = businesses_filtered_count.rename(columns={"keyword": "keywords_matched"})
# Normalize the count of keywords matched
min_count = businesses_filtered_count["keywords_matched"].min()
max_count = businesses_filtered_count["keywords_matched"].max()
businesses_filtered_count["norm_keywords_matched"] = (((businesses_filtered_count["keywords_matched"] - min_count) / (max_count - min_count))) 
# Sort the dataframe based on the normalized count of keywords matched
businesses_filtered_count = businesses_filtered_count.sort_values(by="norm_keywords_matched", ascending=False)

In [ ]:
businesses = businesses.merge(businesses_filtered_count, on="business_id", how="left")
# fill na with 0
businesses["norm_keywords_matched"] = businesses["norm_keywords_matched"].fillna(0)
businesses.head()

# Final Aggregation

In [ ]:
score_weight = 1
keywords_weight = 1
businesses["final_score"] = (businesses["norm_score"] * score_weight) + (
    businesses["norm_keywords_matched"] * keywords_weight
)

In [ ]:
# Print out the top k businesses
businesses.sort_values(by="final_score", ascending=False).head(query["k"])[["name", "stars", "latitude", "longitude", "distance"]]

And these are our top K results